### 線形計画法を用いたMDPにおけるプランニング

* MDPの完全なモデルを分かっている上でのプランニングを考えます。
* 割引無限MDPでは、価値反復と方策反復の計算量は$\gamma$に依存するため、厳密には多項式時間で解くことができません。
  * 多項式時間で解けるというのはアルゴリズムにおけるMDPの入力サイズ($S$、$A$)にのみ多項式的に依存した計算量であることと同義です。
* 線形計画法(以降LP)を用いて完全に多項式時間のアルゴリズムを示したいというのが今回のモチベーションです。


#### Primal LP
$$
min \sum_s\mu(s)V(s)\\

sub V(s) \geq r(s,a) + \gamma\sum_s P(s'|s,a)V(s')
$$

* これを解いてみましょう

In [9]:
import numpy as np
S = 2
A = 3
V = np.zeros(S)
#uniform array dimention is S
mu = np.full(S,1/S)
r = np.random.rand(S*A)
gamma = 0.9
P = np.random.rand(S*A, S)
P = P / P.sum(axis=1)[:, None]

# primaly LP 
from scipy.optimize import linprog

c = mu #係数行列

A_ub = np.zeros((S*A, S))  
b_ub = np.zeros(S*A)

for s in range(S):
    for a in range(A):
        idx = s*A + a
        A_ub[idx,s] = -1
        b_ub[idx] = -r[idx]
        b_ub[idx] -= gamma * np.dot(P[idx], V) 

res = linprog(c, A_ub=A_ub, b_ub=b_ub,bounds=(None, None))

V_optimal = res.x

print(V_optimal)


[0.58818094 0.47200235]


#### 双対LPによる解法

いかのような最適化問題を考えます。
$$
\begin{aligned}
\max & \frac{1}{1-\gamma} \sum_{s, a} d_\mu(s, a) r(s, a) \\
\text { subject to } & d \in \mathcal{K}_\mu
\end{aligned}
$$

なお、

$$
\mathcal{K}_\mu:=\left\{d \mid d \geq 0 \text { and } \sum_a d(s, a)=(1-\gamma) \mu(s)+\gamma \sum_{s^{\prime}, a^{\prime}} P\left(s \mid s^{\prime}, a^{\prime}\right) d\left(s^{\prime}, a^{\prime}\right)\right\}
$$

$$
d_{s_0}^\pi(s, a):=(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \operatorname{Pr}^\pi\left(s_t=s, a_t=a \mid s_0\right)
$$

とします。

In [14]:
#計算がむずいのでPを書き換えます。
P = np.random.rand(S,A,S)
P = P / P.sum(axis = 2, keepdims = True)

d_dimention = S*A
A_eq = np.zeros((S, d_dimention))
b_eq = np.zeros(S)

for s in range(S):
    for a in range(A):
        index = s*A + a

        A_eq[s, index] = 1

        for s_ in range(S):
            for a_ in range(A):
                index_ = s_*A + a_
                A_eq[s, index_] -= gamma * P[s, a_, s_]
    b_eq[s] = (1 - gamma) * mu[s]

c = -r.flatten() / (1 - gamma) 
bounds = [(0, None) for _ in range(d_dimention)]


result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')


if result.success:
    d_star = result.x.reshape(S, A)
    
    pi_star = np.zeros((S, A))
    for s in range(S):
        pi_star[s, :] = d_star[s, :] / d_star[s, :].sum()

    print("Optimal State-Action Visit Distribution (d*):")
    print(d_star)

    print("\nOptimal Policy (π*):")
    print(pi_star)
else:
    print("Optimization failed")
    print("Status:", result.status)
    print("Message:", result.message)

Optimization failed
Status: 2
Message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Basic)
